<a href="https://colab.research.google.com/github/Susheel06/BDA_Assignment_2/blob/main/Big_Data_Analytics_Assignment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import operator
from collections import defaultdict
from itertools import combinations, chain

In [ ]:
!git clone https://github.com/Susheel06/BDA_Assignment_2.git

Cloning into 'BDA_Assignment_2'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [ ]:
class Apriori:

    def __init__(self, minSupport):
        self.support_count = defaultdict(int)
        self.minSupport = minSupport

    def read_transactions_from_file(self, transaction_file):
        with open(transaction_file, "r") as infile:
            transactions = [set(line.rstrip("\n").split(";"))
                            for line in infile]

            return transactions

    def get_one_itemset(self, transactions):
        one_itemset = set()
        for transaction in transactions:
            for item in transaction:
                one_itemset.add(frozenset([item]))

        return one_itemset

    def self_cross(self, Ck, itemset_size):
        Ck_plus_1 = {itemset1.union(itemset2)
                     for itemset1 in Ck for itemset2 in Ck
                     if len(itemset1.union(itemset2)) == itemset_size}
        return Ck_plus_1

    def prune_Ck(self, Ck, Lk_minus_1, itemset_size):
        Ck_ = set()
        for itemset in Ck:
            Ck_minus_1 = list(combinations(itemset, itemset_size-1))
            flag = 0
            for subset in Ck_minus_1:
                if not frozenset(subset) in Lk_minus_1:
                    flag = 1
                    break
            if flag == 0:
                Ck_.add(itemset)
        return Ck_

    def get_min_supp_itemsets(self, Ck, transactions):
        temp_freq = defaultdict(int)

        # update support count of each itemset
        for transaction in transactions:
            for itemset in Ck:
                if itemset.issubset(transaction):
                    temp_freq[itemset] += 1
                    self.support_count[itemset] += 1

        N = len(transactions)
        Lk = [itemset for itemset, freq in temp_freq.items()
              if freq/N > self.minSupport]
        return set(Lk)

    def frequent_item_set(self, transactions):
        K_itemsets = dict()
        Ck = self.get_one_itemset(transactions)
        Lk = self.get_min_supp_itemsets(Ck, transactions)
        k = 2
        while len(Lk) != 0:
            K_itemsets[k-1] = Lk
            Ck = self.self_cross(Lk, k)
            Ck = self.prune_Ck(Ck, Lk, k)
            Lk = self.get_min_supp_itemsets(Ck, transactions)
            k += 1

        return K_itemsets

    def subsets(self, iterable):
        list_ = list(iterable)
        subsets_ = chain.from_iterable(combinations(list_, len)
                                       for len in range(len(list_)+1))
        subsets_ = list(map(frozenset, subsets_))

        return subsets_

    def write_part_1(self, K_itemsets):
        main_dir = "/content/BDA_Assignment_2"
        if not os.path.exists(main_dir):
            os.makedirs(main_dir)

        outfile_path = "/content/BDA_Assignment_2/patterns.txt"
        with open(outfile_path, "w") as outfile:
            for key, values in K_itemsets.items():
                if key > 1:
                    break
                for value in values:
                    support_ct = self.support_count[value]
                    outfile.write("{support}:{label}\n".format(
                        support=support_ct,
                        label=";".join(list(value))
                    ))

    def write_part_2(self, K_itemsets):
        main_dir = '/content/BDA_Assignment_2'
        if not os.path.exists(main_dir):
            os.makedirs(main_dir)

        outfile_path = "/content/BDA_Assignment_2/patterns.txt"
        with open(outfile_path, "w") as outfile:
            for key, values in K_itemsets.items():
                for value in values:
                    support_ct = self.support_count[value]
                    outfile.write("{support}:{label}\n".format(
                        support=support_ct,
                        label=";".join(list(value))
                    ))


if __name__ == "__main__":
    in_transaction_file = "/content/BDA_Assignment_2/categories.txt.txt"

    ap = Apriori(minSupport=0.01)
    transactions = ap.read_transactions_from_file(in_transaction_file)
    K_itemsets = ap.frequent_item_set(transactions)
    ap.write_part_1(K_itemsets)
    ap.write_part_2(K_itemsets)